In [28]:
from PyPDF2 import PdfReader
import re
import json

In [23]:
reader = PdfReader('info.pdf')

In [24]:
print(len(reader.pages))

9


In [26]:
text = ""

page = reader.pages[0]
text = page.extract_text()

# print(text)

In [27]:
raw_text = """
1. Let us consider the algorithm  F(x), where  x is a natural number (1 ≤ x ≤ 106): 
 
Algorithm  F(x):  
    If x = 0 then 
        Return 0 
    Else 
        If x MOD 3 = 0 then 
            Return F(x DIV 10) + 1  
        Else 
            Return F(x DIV 10) 
        EndIf  
    EndIf  
EndAlgorithm  Which of the following function calls will return 4?  
A. F(21369)  
B. F(6933)  
C. F(4) 
D. F(16639) 
"""

In [35]:
def parse_text(text):
    # we get the first part of the question
    # until the word algorithm
    question_match = re.search(r'(.*?)(?=Algorithm)', text, re.DOTALL)
    question_before = question_match.group(1).strip() if question_match else ''

    # we get the code, from algo-endalgo
    code_match = re.search(r'(Algorithm.*?EndAlgorithm)', text, re.DOTALL)
    code = code_match.group(1).strip() if code_match else ''

    # leftover question, after code
    question_after_match = re.search(r'EndAlgorithm(.*?)(?=[A-Z]\.|$)', text, re.DOTALL)
    question_after = question_after_match.group(1).strip() if question_after_match else ''

    question = " ".join([question_before, question_after])

     # answers
    answers_match = re.findall(r'([A-Z]\.(.*?))(?=[A-Z]\.|$)', text, re.DOTALL)
    answers = [{key.strip(): value.strip()} for key, value in answers_match] if answers_match else []

    # Create JSON object
    parsed_json = {
        "question": question,
        "answers": answers,
        "code": code
    }

    # ensure_ascii so we have ≤ instead of u2264
    return json.dumps(parsed_json, indent=4, ensure_ascii=False)


In [38]:
parsed_json = parse_text(raw_text)
print(parsed_json)

{
    "question": "1. Let us consider the algorithm  F(x), where  x is a natural number (1 ≤ x ≤ 106): Which of the following function calls will return 4?",
    "answers": [
        {
            "A. F(21369)": "F(21369)"
        },
        {
            "B. F(6933)": "F(6933)"
        },
        {
            "C. F(4)": "F(4)"
        },
        {
            "D. F(16639)": "F(16639)"
        }
    ],
    "code": "Algorithm  F(x):  \n    If x = 0 then \n        Return 0 \n    Else \n        If x MOD 3 = 0 then \n            Return F(x DIV 10) + 1  \n        Else \n            Return F(x DIV 10) \n        EndIf  \n    EndIf  \nEndAlgorithm"
}
